In [34]:
import sys
import os
path = os.path.abspath('')
path = os.path.dirname(path)
sys.path.insert(1, str(path))


In [35]:

from datetime import date
import os
import load_csv

data_frame = load_csv.load(path + "/data/clean_data.csv")
context = load_csv.getContext()

In [37]:
data_frame.show()

+--------------------+----------------+-----------------+----------+--------------------+-------------------+----------------+---+------+----------+--------------------+----------------+----------------------+----------------+-------------------+
|            order_id|     customer_id|    customer_name|product_id|        product_name|   product_category|    payment_type|qty| price|  datetime|             country|            city|ecommerce_website_name|  payment_txn_id|payment_txn_success|
+--------------------+----------------+-----------------+----------+--------------------+-------------------+----------------+---+------+----------+--------------------+----------------+----------------------+----------------+-------------------+
|2NEBnZETy1FdFpKJf...|wEwuo1GDdnbC8ju5|   STEPHEN THOMAS|     14315|ESR Boost Kicksta...|tv, audio & cameras|             UPI|  3| 25.19|2022-05-19|       United States|       Blackhawk|       www.wayfair.com|XXFqXfizoOtsQ0S1|                  Y|
|cs4mesS4QiK

In [38]:
data_frame.createOrReplaceTempView('orders')

In [47]:
def get_popularity_over_year():
    query_split_datetime = 'CONCAT(YEAR(datetime), \'_Q\',QUARTER(datetime)) AS year_quarter'
    query_quarters = f'SELECT {query_split_datetime} FROM orders'
    query_unique_quarters = f'SELECT DISTINCT year_quarter FROM ({query_quarters}) ORDER BY year_quarter'
    quarters_df = context.sql(query_unique_quarters)
    quarter_list = quarters_df.collect()
    quarter_list = [row['year_quarter'] for row in quarter_list]

    query_pivot = '('
    for index, quarter in enumerate(quarter_list):
        if index == len(quarter_list) - 1:
            query_pivot += f'\'{quarter}\')'
        else:
            query_pivot += f'\'{quarter}\', '

    query_popularity_over_year_subquery = f'SELECT {query_split_datetime}, product_id, product_name, qty FROM orders'
    query_popularity_over_year = f'SELECT * FROM ({query_popularity_over_year_subquery}) PIVOT (SUM(qty) AS popularity FOR year_quarter IN {query_pivot})'
    pop_year_df = context.sql(query_popularity_over_year)
    pop_year_df = pop_year_df.na.fill(0)
    for index, quarter in enumerate(quarter_list):
        pop_year_df.sort(quarter, ascending=False).select(['product_id', 'product_name', quarter]).show(10)

    
    # pop_year_df.show(1000)
    return pop_year_df

def get_popularity_over_country():
    query_unique_countries = f'SELECT DISTINCT country FROM orders ORDER BY country'
    countries_df = context.sql(query_unique_countries)
    country_list = countries_df.collect()
    country_list = [row['country'] for row in country_list]

    query_pivot = '('
    for index, country in enumerate(country_list):
        if index == len(country_list) - 1:
            query_pivot += f'\'{country}\')'
        else:
            query_pivot += f'\'{country}\', '

    query_popularity_over_country_subquery = f'SELECT country, product_id, product_name, qty FROM orders'
    query_popularity_over_country = f'SELECT * FROM ({query_popularity_over_country_subquery}) PIVOT (SUM(qty) AS popularity FOR country IN {query_pivot})'

    pop_country_df = context.sql(query_popularity_over_country)
    pop_country_df = pop_country_df.na.fill(0)
    
    for index, country in enumerate(country_list):
        pop_country_df.sort(country, ascending=False).select(['product_id', 'product_name', country]).show(10)

    # pop_country_df.show(1000)
    return pop_country_df

pop_year_df = get_popularity_over_year()
pop_country_df = get_popularity_over_country()

+----------+--------------------+-------+
|product_id|        product_name|2022_Q1|
+----------+--------------------+-------+
|    130653|AnjuShree Choice ...|     49|
|    137496|Vaamsi Women's Po...|     41|
|    211788|KLEIO Mutli Slot ...|     39|
|    532537|Flingr Poly Spand...|     34|
|    514428|RR SA KURTIS Wome...|     34|
|    230681|A2Z Shop In-Ear H...|     30|
|    148624|Yellow Chimes Ank...|     30|
|    384880|Philips Avent Cla...|     30|
|    371737|SEGA Primer 2023 ...|     29|
|    288767|Pepe Mens 5 Pocke...|     29|
+----------+--------------------+-------+
only showing top 10 rows

+----------+--------------------+-------+
|product_id|        product_name|2022_Q2|
+----------+--------------------+-------+
|     20682|Sixfire® Cleaner ...|     41|
|    384439|Mee Mee 2 in 1 Sp...|     39|
|    152796|ZAVERI PEARLS Ros...|     39|
|    164841|Sukkhi Glimmery G...|     36|
|    279443|DAMENSCH Deo-Soft...|     34|
|    451340|FORTIVA Stylish R...|     34|
|     42

In [98]:

# for jupyter
path1 = path + "/data/miguel/popularity_over_year.csv"
path2 = path + "/data/miguel/popularity_over_country.csv"
pop_year_df.toPandas().to_csv(str(path1), index=False)
pop_country_df.toPandas().to_csv(str(path2), index=False)